# 1. Data Preparation
Resave data as a set of tiff files in order to match conventions expected by TracX.

In [1]:
import os

import numpy as np
from tifffile import imwrite

from deepcell_tracking.trk_io import load_trks

In [13]:
source_data = '../../data/test.trks'
data_dir = 'data'
raw_dir = os.path.join(data_dir, 'raw')
seg_dir = os.path.join(data_dir, 'seg')

for d in [raw_dir, seg_dir]:
    if not os.path.exists(d):
        os.makedirs(d)

Load the test split of the tracking data

In [3]:
data = load_trks(source_data)

In [5]:
def find_zero_padding(X):
    """Remove zero padding to avoid adverse effects on model performance"""
    # Calculate position of padding based on first frame
    # Assume that padding is in blocks on the edges of image
    good_rows = np.where(X[0].any(axis=0))[0]
    good_cols = np.where(X[0].any(axis=1))[0]

    slc = (
        slice(None),
        slice(good_cols[0], good_cols[-1] + 1),
        slice(good_rows[0], good_rows[-1] + 1),
        slice(None)
    )

    return slc

In [8]:
# ChannelName_positionXXYYZZZ_timeTTTT.tif
name_template = '{}_position{:02}{:02}000_time{:05}.tif'

In [16]:
for batch_no in range(len(data['lineages'])):
    # Build subdirectories for data
    raw_subdir = os.path.join(raw_dir, '{:03}'.format(batch_no + 1))
    seg_subdir = os.path.join(seg_dir, '{:03}'.format(batch_no + 1))
    
    # Create directories if needed
    for d in (raw_subdir, seg_subdir):
        if not os.path.exists(d):
            os.makedirs(d)
    
    # Pull out relevant data for this batch
    X = data['X'][batch_no]
    y = data['y'][batch_no]
    lineages = data['lineages'][batch_no]
    
    # Determine position of zero padding for removal
    slc = find_zero_padding(X)
    X = X[slc]
    y = y[slc]
    
    # Determine which frames are zero padding
    frames = np.sum(y, axis=(1,2)) # True if image not blank
    good_frames = np.where(frames)[0]
    # We assume here that the empty frames are at the end of the movie (padding rather than skipped)
    movie_len = len(good_frames)
    
    # Position info for naming convention
    x_pos = batch_no + 1
    y_pos = 1
    
    # Save each frame of the movie as an individual tif
    channel = 0 # These images should only have one channel
    for i in range(movie_len):
        name_raw = os.path.join(raw_subdir, name_template.format('nuclear', x_pos, y_pos, i))
        name_mask = os.path.join(seg_subdir, name_template.format('mask', x_pos, y_pos, i))
        
        imwrite(name_raw, X[i, ..., channel])
        imwrite(name_mask, y[i, ..., channel].astype('uint16'))

# 2. Tracking
1. Open Matlab > 2020 and open the TracX folder
2. From the Matlab terminal, run the following 
```
addpath(genpath('tracx'));
Tracker = TracX.Tracker();
TracX.GUI.TracXGUI();
```
3. 